<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [143]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric
import torch
import torch.nn as nn
import torch.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

import collections
import random

# Load a dataset and print the first examples in the training set
# wikibio = load_dataset('wiki_bio', cache_dir="/Volumes/External HD/Dev/datasets/wikibio", split='train[:1%]')
# wikibio[0]






In [ ]:
wikitext = load_dataset('wikitext', 'wikitext-103-raw-v1', cache_dir="/Volumes/External HD/Dev/datasets/wikitext", split='train[:1%]')

In [ ]:

# Process the dataset - add a column with the length of the context texts
dataset_with_length = squad_dataset.map(lambda x: {"length": len(x["context"])})

# Process the dataset - tokenize the context texts (using a tokenizer from the 🤗 transformers library)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenized_dataset = squad_dataset.map(lambda x: tokenizer(x['context']), batched=True)

In [3]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')

# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])


In [6]:
tokens_tensor

tensor([[ 8241,   373,  5395,   367, 19069,  5633,  5395,   367, 19069,   373,
           257]])

In [2]:
import spacy
# from spacy.pipeline.ner import DEFAULT_NER_MODEL

nlp = spacy.load(
    "en_core_web_sm", 
    exclude=['tagger', 'parser', 'attribute_ruler', 'lemmatizer']
) #Note: use larger model for production
nlp.add_pipe('sentencizer')

[('San Francisco', 0, 13, 'GPE')]
['San', 'B', 'GPE']
['Francisco', 'I', 'GPE']


In [11]:
# !python -m spacy download en_core_web_sm

When processing large volumes of text, the statistical models are usually more efficient if you let them work on batches of texts. spaCy’s nlp.pipe method takes an iterable of texts and yields processed Doc objects. The batching is done internally.

In [42]:
sample_iterable = [wikitext['text'][-i] for i in range(20) if len(wikitext['text'][-i]) > 40]

In [48]:
sample_iterable[0]

" During the 1990s WNO made its Proms debut , with a complete performance of Mozart 's Idomeneo , conducted by Mackerras in 1991 . The company played three short seasons at the Royal Opera House , Covent Garden in the mid @-@ 1990s , featuring Tristan und Isolde and La favorita in 1993 , The Yeomen of the Guard in 1995 , and The Rake 's Progress and the jubilee double bill of Cavalleria Rusticana and Pagliacci in 1996 . \n"

In [85]:
def runNER(model, texts):
    for doc in nlp.pipe(texts):
        for sent in doc.sents:
            if sent.ents:
                ents = str([(e.text, e.start_char - sent.start_char, e.end_char - sent.start_char, e.label_) for e in sent.ents]).strip('[]')
                out = f"{sent.text}|{ents}"
                print(out)
                print("\n")

In [86]:
runNER(nlp, sample_iterable)

 During the 1990s WNO made its Proms debut , with a complete performance of Mozart 's Idomeneo , conducted by Mackerras in 1991 .|('the 1990s', 8, 17, 'DATE'), ('WNO', 18, 21, 'ORG'), ('Proms', 31, 36, 'ORG'), ('Mozart', 76, 82, 'PERSON'), ('Idomeneo', 86, 94, 'PERSON'), ('Mackerras', 110, 119, 'ORG'), ('1991', 123, 127, 'DATE')


The company played three short seasons at the Royal Opera House , Covent Garden in the mid @-@ 1990s , featuring Tristan und Isolde and La favorita in 1993 , The Yeomen of the Guard in 1995 , and The Rake 's Progress and the jubilee double bill of Cavalleria Rusticana and Pagliacci in 1996 .|('three', 19, 24, 'CARDINAL'), ('the Royal Opera House', 42, 63, 'ORG'), ('Covent Garden', 66, 79, 'FAC'), ('the mid @-@ 1990s', 83, 100, 'DATE'), ('Tristan', 113, 120, 'GPE'), ('Isolde', 125, 131, 'GPE'), ('La', 136, 138, 'GPE'), ('1993', 151, 155, 'DATE'), ('The Yeomen of the Guard', 158, 181, 'WORK_OF_ART'), ('1995', 185, 189, 'DATE'), ("The Rake 's Progress", 196, 216

In [145]:
class DataProcessor:
    def __init__(
        self, 
        text, 
        write_dir=None, 
        n_workers=4
    ):
        self.text = text
        self.write_dir = write_dir
        self.n_workers = n_workers
        
        self.ner_texts = []
        self.ents = collections.defaultdict(list)
        self.permuted = []

        self.model = spacy.load(
            "en_core_web_sm", 
            exclude=['tagger', 'parser', 'attribute_ruler', 'lemmatizer']
        )
        self.model.add_pipe('sentencizer')
        
        self.keep_ents = ['PERSON', 'ORG', 'GPE']
        
    @classmethod
    def fromFile(cls, file_loc):
        pass
        
    
    def permuteEnts(self):
        timestamp = time.time()
        if self.write_dir:
            handler = open(self.write_dir + f'/permuted_entities.{timestamp}', 'w')
        for (sent, ents) in self.ner_texts:
            eligible = list(filter(lambda x: x[3] in self.keep_ents, ents))
            orig_ent = random.choice(eligible)
            ent_type = orig_ent[3]
            start, end  = orig_ent[1:3]
            while True:
                replace_ent = random.choice(self.ents[ent_type])
                if replace_ent != orig_ent: break

            prefix = sent[:start]
            suffix = sent[end:]
            new_sent = prefix + replace_ent + suffix
            if self.write_dir:
                handler.write(new_sent + "\n")
            self.permuted.append(new_sent)
    
    
    def processEnts(self):
        timestamp = time.time()
        
        if self.write_dir:
            main_loc = self.write_dir + f'/processed_entities.{timestamp}'
            ent_loc = self.write_dir + f'/all_entities.{timestamp}'
            with open(main_loc, 'w') as f:
                for output in self.runNER(self.text):
                    text, ents = output
                    f.write(f"{text}|{str(ents).strip('[]')}\n")
            with open(ent_loc, 'w') as f:
                for ent_type, ent_value in self.ents.items():
                    f.write(f"{ent_type}|{ent_value}\n")
        else:
            for output in self.runNER(self.text):
                self.ner_texts.append(output)
        
        
    def runNER(self, texts):
        for doc in self.model.pipe(texts):
            for sent in doc.sents:
                if any([e.label_ in self.keep_ents for e in sent.ents]):
                    ents = []
                    for e in sent.ents:
                        ents.append((e.text, e.start_char - sent.start_char, e.end_char - sent.start_char, e.label_))
                        self.ents[e.label_].append(e.text)
                    yield (sent.text, ents)
    

In [146]:
dp = DataProcessor(wikitext['text'][1:1000])

In [147]:
dp.processEnts()

In [148]:
dp.permuteEnts()

In [158]:
idxs = random.sample(range(1, 900), 10)
for idx in idxs:
    print(dp.ner_texts[idx][0])
    print(dp.permuted[idx])
    print('\n')

Horus was the most important god in the Early Dynastic Period , Ra rose to preeminence in the Old Kingdom , Amun was supreme in the New , and in the Ptolemaic and Roman periods , Isis was the divine queen and creator goddess .
Horus was the most important god in the Early Dynastic Period , Ra rose to preeminence in the Old Kingdom , Amun was supreme in the New , and in the Ptolemaic and Roman periods , CAA was the divine queen and creator goddess .


Due to its association with the Camden Expedition of 1864 , the arsenal may be included in the Camden Expedition Sites National Historic Landmark designated in 1994 .
Due to its association with GA of 1864 , the arsenal may be included in the Camden Expedition Sites National Historic Landmark designated in 1994 .


The win qualified Gambia for the 2012 Azerbaijan World Cup .
The win qualified Dorothy for the 2012 Azerbaijan World Cup .


His father and uncles Brian and Ray all played in the NHL , and while Cullen and his three brothers all